In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from itertools import combinations
import statsmodels.api as sm
lowess = sm.nonparametric.lowess
# import matplotlib.pyplot as plt
# from matplotlib_venn import venn2
import re
import warnings
import pybedtools
from pybedtools import BedTool
import os
warnings.filterwarnings('ignore')

In [2]:
chr_list = [
    "chr1", "chr2", "chr3", "chr4", "chr5", "chr6",
    "chr7", "chr8", "chr9", "chr10", "chr11", "chr12",
    "chr13", "chr14", "chr15", "chr16", "chr17", "chr18",
    "chr19", "chr20", "chr21", "chr22", "chrX",
]

In [3]:
def load_fhap(path):
    """laoding phasing fragments"""
    fhap_list=path + '/' + 'frag-hap_mapq5.list'
    fhap=pd.read_csv(fhap_list,header=None,sep='\t',names=['rid','fid','cid','pos','hp'],
                    usecols = [0,1,2,3,5])
    print(len(fhap))

    ### read-level phasing stats
    read_df = fhap.groupby('rid')['hp'].apply(list).reset_index()
    read_df['h1'] = read_df.hp.apply(lambda x: x.count(1) if 1 in x else 0)
    read_df['h2'] = read_df.hp.apply(lambda x: x.count(2) if 2 in x else 0)
    read_df['un'] = read_df.hp.apply(lambda x: x.count(0) if 0 in x else 0)
    read_df['hp'] = read_df['h1'] + read_df['h2']
    read_df['fc'] = read_df['hp'] + read_df['un']
    read_df['mod_fc'] = read_df.fc.apply(lambda x: x if x<=10 else 10)
    read_df['ratio'] = read_df.apply(lambda x: round(x.hp/x.fc*100,2),axis=1)
    ##### read haplotype assign
    read_df["HType"] = "h-trans"
    P1 = read_df.h1 == read_df.hp
    P2 = read_df.h2 == read_df.hp
    P3 = read_df.ratio == 0
    read_df.loc[P1, "HType"] = "h1"
    read_df.loc[P2, "HType"] = "h2"
    read_df.loc[P3, "HType"] = "unknown"
    read_df = read_df.loc[read_df.fc >= 5]
    
    return read_df, fhap

In [4]:
def filter_hp(read_df, fhap, FLANK):
    """filter reads with haplotag h1 or h2, and set the bin size to 5kb"""
    flanksize = FLANK
    # Filter hp reads
    h1_reads = read_df.loc[read_df.HType=="h1", "rid"].values
    h2_reads = read_df.loc[read_df.HType=="h2", "rid"].values
    fhap["HType"] = "unknown"
    fhap.loc[fhap.rid.isin(h1_reads), "HType"] = "h1"
    fhap.loc[fhap.rid.isin(h2_reads), "HType"] = "h2"
    fhap_filter = fhap.loc[fhap.HType!="unknown", :].copy()
    # fragment flank as 5kb length
    fhap_filter["start"] = fhap_filter["pos"] - flanksize
    fhap_filter["end"] = fhap_filter["pos"] + flanksize
    
    return fhap_filter

In [5]:
def LoadingPro(filename, region):#, binsize
    print("Loading %s"%filename)
    bedDF = pd.read_csv(filename, sep="\t", header=None, 
                        usecols=[0,1,2,6], names=["Pchrom","Pstart","Pend","ID"])
    Pchr = bedDF.Pchrom == region[0]
    Pregion = (bedDF.Pstart >= region[1]) & (bedDF.Pend <= region[2])
    bedDF = bedDF.loc[Pchr&Pregion,:]
    return(bedDF)

In [6]:
def inter(pbed, fhap_filter):
    """pro, frag intersect"""
    
    pbed = BedTool.from_dataframe(pro_bed)
    fbed = BedTool.from_dataframe(fhap_filter)

    intersect = fbed.intersect(pbed, wa=True, wb=True, loj=True)
    # intersect2 = intersect.intersect(fbed, wa=True, wb=True, loj=True)

    colnames = ["chrom", "start", "end", "rid", "fid","cid", "pos", "hp","HType",
                "Pchrom", "Pstart", "Pend", "ID"]
    int_df = intersect.to_dataframe(names=colnames)

    pybedtools.cleanup() 
    
    
    return int_df

In [7]:
def func(ID, df):
    """contact matrix make processing"""

    rid_list = list(df['rid'].unique())
    tmp_gene_list = []
    for rid in rid_list:
        tmp_df = rid_dict[rid]
        tmp_df['Pchrom'] = df.loc[0, 'Pchrom']
        tmp_df['Pstart'] = df.loc[0, 'Pstart']
        tmp_df['Pend'] = df.loc[0, 'Pend']
        tmp_df['ID'] = df.loc[0, 'ID']
        tmp_gene_list.append(tmp_df)

    return pd.concat(tmp_gene_list)
import multiprocessing
def func1_wrapper(args):
    """pass parameter"""
    return func(*args)
def file_tackle(df):
    """make DF"""
    print("make DF")
    
#     Pid_rid_group = int_df_filter.groupby(['rid'])
#     Pid_rid_dict = {}
#     for group, df in Pid_rid_group:
#         Pid_rid_dict[group] = df

    
    
    gene_group = int_df.groupby("ID")
    gene_dict = {}
    for ID, df in gene_group:
        gene_dict[ID] = df.reset_index(drop = True)
    del gene_dict['.']
    if __name__ == '__main__':
        with multiprocessing.Pool(processes=20) as pool:
            args_list = [(key, value) for key, value in gene_dict.items()]
            result = pool.map(func1_wrapper, args_list)
    DF = pd.concat(result)
    DF["fpos"] = ( DF['start'] + DF['end'] ) / 2
    DF["ppos"] = ( DF['Pstart'] + DF['Pend'] ) / 2
    DF["dis"] = abs(( DF['ppos'] - DF['fpos'] ))
    
    return DF

In [8]:
def dis_loc_count(pid, df):
    """Cal DLR"""
    concat_list = []
    for read_id, df in df.groupby(by = "rid"):
        
        tmp_list = []
        tmp_h1 = df.loc[df.HType == "h1"]
        distal1 = len(tmp_h1.loc[tmp_h1.dis >= 1500000])
        local1 = len(tmp_h1.loc[tmp_h1.dis < 1500000])

        tmp_h2 = df.loc[df.HType == "h2"]
        distal2 = len(tmp_h2.loc[tmp_h2.dis >= 1500000])
        local2 = len(tmp_h2.loc[tmp_h2.dis < 1500000])

        tmp_list.append(pid)
        tmp_list.append(read_id)
        tmp_list.append(distal1)
        tmp_list.append(local1)
        tmp_list.append(distal2)
        tmp_list.append(local2)
        concat_list.append(pd.DataFrame(tmp_list).T)
    concat_df = pd.concat(concat_list).rename(columns =
                                              {0:"Pid",
                                               1:"read_id",
                                               2:"dis_h1",
                                               3:"loc_h1",
                                               4:"dis_h2",
                                               5:"loc_h2"}).set_index(["Pid","read_id"])
        
    return concat_df
import multiprocessing
def func2_wrapper(args):
    """pass parameter"""
    return dis_loc_count(*args)
def computing_dlr(DF):
    """calculate the singel melecular Distal and Primary ratio"""
    final_dict = {}
    DF_group = DF.groupby("ID")
    for pid, df in DF_group:
        final_dict[pid] = df
    print("Calculating ...")
    if __name__ == '__main__':
        with multiprocessing.Pool(processes=20) as pool:
            args_list = [(key, value) for key, value in final_dict.items()]
            result = pool.map(func2_wrapper, args_list)
    DF_statistic = pd.concat(result)
    return DF_statistic

In [9]:
def math_div_plus(DF_result_copy):
    """h1_DLR"""
    a1 = "dis_h1"
    b1 = "loc_h1"
    c1 = "DLR_h1"

    DF_result_copy.loc[(DF_result_copy[f'{a1}'] == 0) & (DF_result_copy[f'{b1}'] ==0), f"{c1}"] = np.nan

    DF_result_copy.loc[(DF_result_copy[f'{a1}'] != 0) & (DF_result_copy[f'{b1}'] !=0), f"{c1}"] = ((DF_result_copy.loc[(DF_result_copy[f'{a1}'] != 0) &
                                                                                                                       (DF_result_copy[f'{b1}'] !=0), f"{a1}"]) /
                                                                                                  (DF_result_copy.loc[(DF_result_copy[f'{a1}'] != 0) &
                                                                                                                      (DF_result_copy[f'{b1}'] !=0), f"{b1}"]))

    DF_result_copy.loc[(DF_result_copy[f'{a1}'] != 0) & (DF_result_copy[f'{b1}'] ==0), f"{c1}"] = DF_result_copy.loc[(DF_result_copy[f'{a1}'] != 0) &
                                                                                                                     (DF_result_copy[f'{b1}'] ==0), f"{a1}"]

    DF_result_copy.loc[(DF_result_copy[f'{a1}'] == 0) & (DF_result_copy[f'{b1}'] !=0), f"{c1}"] = 0
    return DF_result_copy

In [10]:
def math_div_negative(DF_result_copy):
    """h2_DLR"""
    a2 = "dis_h2"
    b2 = "loc_h2"
    c2 = "DLR_h2"
    
    DF_result_copy.loc[(DF_result_copy[f'{a2}'] == 0) 
                       & (DF_result_copy[f'{b2}'] ==0), f"{c2}"] = np.nan

    DF_result_copy.loc[(DF_result_copy[f'{a2}'] != 0) 
                       & (DF_result_copy[f'{b2}'] !=0), f"{c2}"] = ((DF_result_copy.loc[(DF_result_copy[f'{a2}'] != 0) 
                                                                                        & (DF_result_copy[f'{b2}'] !=0), f"{a2}"]) 
                                                                    /(DF_result_copy.loc[(DF_result_copy[f'{a2}'] != 0) 
                                                                                        & (DF_result_copy[f'{b2}'] !=0), f"{b2}"]))

    DF_result_copy.loc[(DF_result_copy[f'{a2}'] != 0)
                       &(DF_result_copy[f'{b2}'] ==0), f"{c2}"] = DF_result_copy.loc[(DF_result_copy[f'{a2}'] != 0)
                                                                                     & (DF_result_copy[f'{b2}'] ==0), f"{a2}"]

    DF_result_copy.loc[(DF_result_copy[f'{a2}'] == 0) & (DF_result_copy[f'{b2}'] !=0), f"{c2}"] = 0
    return DF_result_copy

In [11]:
def mean_DLR(statistic):
    """Calculate the mean SDPR value per gene promoter"""
    statistic_group1 = statistic.groupby(by=["Pid"])["DLR_h1"]

    groupdict1 = {}
    for g in statistic_group1:
        key = g[0]
        groupdict1[key] = [ g[1].mean(), g[1].std(), len(g[1].dropna()) ]

    statistic_group2 = statistic.groupby(by=["Pid"])["DLR_h2"]

    groupdict2 = {}
    for g in statistic_group2:
        key = g[0]
        groupdict2[key] = [ g[1].mean(), g[1].std(), len(g[1].dropna()) ]
    h1_statistic = pd.DataFrame(groupdict1, index = ['h1_mean', 'h1_std', 'h1_nobs']).T
    h2_statistic = pd.DataFrame(groupdict2, index = ['h2_mean', 'h2_std', 'h2_nobs']).T
    
    ttest_df = pd.merge(h1_statistic.reset_index(), h2_statistic.reset_index(), on = "index")
    
    return ttest_df

In [12]:
chrom_size = pd.read_csv(f"{Rawdir}/pre_data/hg38.chromosomes.size",
                        sep = "\t", header = None,
                        names = ['chrom', 'size'])


In [ ]:
PATH = f"{Rawdir}/fhap_list"
OUTPATH = f"{Rawdir}/SDPR"
for file in chr_list:
    #chromosome size
    SIZE = chrom_size.loc[chrom_size.chrom == file, 'size'].tolist()[0]
    region = [file, 0, SIZE]
    
    #reading TSS promoter
    promoterfile = f"{Rawdir}/pre_data/TSS_sorted.txt"
    pro_bed =  LoadingPro(promoterfile, region)#, binsize
    
    #loading
    path = PATH + '/' + file
    print(f"Loading {path}")
    read_df, fhap = load_fhap(path)
    break
    
    fhap_filter = filter_hp(read_df, fhap, 500)
    fhap_filter['chrom'] = file
    fhap_filter = fhap_filter[['chrom', 'start', 'end', 'rid', 'fid', 'cid', 'pos', 'hp', 'HType',  ]]
    int_df = inter(pro_bed, fhap_filter)
    rid_group = fhap_filter.groupby("rid")
    rid_dict = {}
    for rid, df in rid_group:
        rid_dict[rid] = df.reset_index(drop = True)

    DF = file_tackle(int_df)
    break

    DF_statistic = computing_dlr(DF)
    DF_statistic = math_div_plus(DF_statistic)
    DF_statistic = math_div_negative(DF_statistic)
    
    #mean SDPR per gene promoter
    ttest_df = mean_DLR(DF_statistic)
    len1 = len(ttest_df)
    ttest_df = ttest_df.dropna()
    len2 = len(ttest_df)
    print(f"filtered {len1 - len2} nan genes")
    from scipy import stats
    ttest_df[['statistic', 'pvalue']] = ttest_df.apply(lambda row: pd.Series(
        stats.ttest_ind_from_stats(mean1=row['h1_mean'],
                                   std1=row['h1_std'],
                                   nobs1=row['h1_nobs'],
                                   mean2=row['h2_mean'],
                                   std2=row['h2_std'],
                                   nobs2=row['h2_nobs'])), axis=1)

    ttest_df["Fold_change"] = np.log2( ttest_df["h1_mean"].values /  ttest_df["h2_mean"].values )
    
    from statsmodels.stats.multitest import multipletests
    # alpha = 0.1
    rejected, p_adjusted, _, alpha_corrected =  multipletests(ttest_df["pvalue"].values, 
                                                              # alpha=alpha, 
                                                              method='fdr_bh', #fdr_bh
                                                              is_sorted=False, 
                                                              returnsorted=False)
    ttest_df["adjpval"] =  p_adjusted
    
    print(f"writing to {OUTPATH}/{file}/DF_mapq5.txt")
    DF.to_csv(OUTPATH + '/' + file + '/' + 'DF_mapq5.txt',
             sep = "\t",index = None)
    print(f"writing to {OUTPATH}/{file}/ttest_df_parallel_mapq5.txt")
    ttest_df.to_csv(OUTPATH + '/' + file + '/' + 'ttest_df_parallel_mapq5.txt',
                     sep = "\t",index = None)
    


    read_df = None
    fhap = None
    fhap_filter = None
    int_df_filter = None
    rid_group = None
    rid_dict = None
    DF = None
    DF_statistic = None
    pro_bed = None
    DF_statistic = None
    ttest_df = None

    print("write done!")